# 0-5: ML 개념 + Sklearn

머신러닝 기초 개념과 Sklearn 사용법을 학습합니다.

## 학습 목표
1. ML 기초 (분류 vs 회귀)
2. 과적합과 train/test split
3. fit → predict 패턴
4. 평가 지표 (Accuracy, Recall, Precision, AUC)
5. 트리 모델 비교 (Decision Tree, Random Forest, XGBoost)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
from xgboost import XGBClassifier

print("✅ 패키지 임포트 완료")

---
## 1. ML이란?

### 📚 분류 vs 회귀

| 유형 | 출력 | 예시 |
|-----|-----|-----|
| **분류 (Classification)** | 범주 (0/1, A/B/C) | 사기 탐지, 스팸 분류 |
| **회귀 (Regression)** | 연속값 | 집값 예측, 매출 예측 |

**FDS는 분류 문제**: 사기(1) vs 정상(0)

---
## 2. 과적합과 데이터 분할

### 📚 왜 train/test를 나누는가?

- **과적합(Overfitting)**: 학습 데이터만 잘 맞추고, 새 데이터에 실패
- 해결: 학습에 안 쓴 데이터(test)로 성능 검증

```
전체 데이터
├── Train (80%) → 모델 학습
└── Test (20%)  → 성능 평가
```

In [ ]:
# 📚 샘플 데이터 생성
np.random.seed(42)
n_samples = 1000

# 특성 (Feature)
amount = np.random.exponential(50000, n_samples)
hour = np.random.randint(0, 24, n_samples)
is_foreign = np.random.randint(0, 2, n_samples)

# 타겟 (Label) - 고액 + 새벽 + 해외 = 사기 확률 높음
fraud_prob = 0.01 + 0.05 * (amount > 100000) + 0.1 * ((hour < 6) | (hour > 22)) + 0.1 * is_foreign
is_fraud = (np.random.random(n_samples) < fraud_prob).astype(int)

# DataFrame
df = pd.DataFrame({
    'amount': amount,
    'hour': hour,
    'is_foreign': is_foreign,
    'is_fraud': is_fraud
})

print(f"데이터 크기: {df.shape}")
print(f"사기 비율: {df['is_fraud'].mean():.2%}")
df.head()

In [ ]:
# 📚 train_test_split
X = df[['amount', 'hour', 'is_foreign']]  # 특성
y = df['is_fraud']                         # 타겟

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% 테스트용
    random_state=42     # 재현성
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

### 💻 실습 1: train_test_split

데이터를 70:30 비율로 분할하세요.

In [ ]:
# 💻 실습 1
# TODO: 70:30 비율로 분할 (test_size=0.3, random_state=42)
X_train_70, X_test_30, y_train_70, y_test_30 = None, None, None, None  # TODO

print(f"Train: {len(X_train_70)}, Test: {len(X_test_30)}")

In [ ]:
# 체크포인트 1
assert len(X_train_70) == 700, "Train 700개"
assert len(X_test_30) == 300, "Test 300개"

print("✅ 체크포인트 1 통과!")

---
## 3. fit → predict 패턴

In [ ]:
# 📚 sklearn 공통 패턴
# 1. 모델 생성
model = DecisionTreeClassifier(random_state=42)

# 2. 학습 (fit)
model.fit(X_train, y_train)

# 3. 예측 (predict)
y_pred = model.predict(X_test)

print("예측 결과 (처음 10개):")
print(y_pred[:10])

In [ ]:
# 📚 predict_proba: 확률 예측
y_proba = model.predict_proba(X_test)

print("확률 예측 (처음 5개):")
print("[정상 확률, 사기 확률]")
print(y_proba[:5])

# 사기 확률만
fraud_prob = y_proba[:, 1]
print(f"\n사기 확률: {fraud_prob[:5]}")

### 💻 실습 2: fit/predict

RandomForest로 학습하고 예측하세요.

In [ ]:
# 💻 실습 2
# TODO: RandomForestClassifier 생성 (n_estimators=100, random_state=42)
rf_model = None  # TODO

# TODO: 학습

# TODO: 예측
rf_pred = None  # TODO

# TODO: 사기 확률 예측
rf_proba = None  # TODO ([:, 1]로 사기 확률만)

print(f"예측: {rf_pred[:10]}")
print(f"확률: {rf_proba[:10]}")

In [ ]:
# 체크포인트 2
assert rf_model is not None, "모델 생성 필요"
assert len(rf_pred) == len(X_test), "예측 개수 일치"
assert 0 <= rf_proba.min() <= rf_proba.max() <= 1, "확률은 0~1"

print("✅ 체크포인트 2 통과!")

---
## 4. 평가 지표

### 📚 분류 평가 지표

| 지표 | 의미 | FDS 관점 |
|-----|-----|--------|
| **Accuracy** | 전체 정확도 | 불균형 데이터에선 무의미 |
| **Recall** | 실제 사기 중 탐지 비율 | **가장 중요** (놓치면 안됨) |
| **Precision** | 예측 사기 중 실제 사기 | 높으면 오탐 적음 |
| **AUC** | 전체 성능 | 0.5=랜덤, 1.0=완벽 |

In [ ]:
# 📚 혼동 행렬 (Confusion Matrix)
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print("혼동 행렬:")
print(cm)
print()
print("         예측=정상  예측=사기")
print(f"실제=정상   {cm[0,0]:5d}    {cm[0,1]:5d}  (TN, FP)")
print(f"실제=사기   {cm[1,0]:5d}    {cm[1,1]:5d}  (FN, TP)")

In [ ]:
# 📚 평가 지표 계산
acc = accuracy_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba[:, 1])

print(f"Accuracy:  {acc:.4f}")
print(f"Recall:    {rec:.4f}  ← 사기 탐지율")
print(f"Precision: {prec:.4f}")
print(f"AUC:       {auc:.4f}")

### 💻 실습 3: 평가 지표 계산

RandomForest 모델의 평가 지표를 계산하세요.

In [ ]:
# 💻 실습 3
# TODO: RandomForest 예측 결과(rf_pred, rf_proba)로 평가 지표 계산
rf_acc = None   # TODO: accuracy_score
rf_rec = None   # TODO: recall_score
rf_prec = None  # TODO: precision_score
rf_auc = None   # TODO: roc_auc_score

print(f"RandomForest 성능:")
print(f"  Accuracy:  {rf_acc:.4f}")
print(f"  Recall:    {rf_rec:.4f}")
print(f"  Precision: {rf_prec:.4f}")
print(f"  AUC:       {rf_auc:.4f}")

In [ ]:
# 체크포인트 3
assert rf_acc is not None, "Accuracy 계산 필요"
assert 0 <= rf_rec <= 1, "Recall은 0~1"
assert 0 <= rf_auc <= 1, "AUC는 0~1"

print("✅ 체크포인트 3 통과!")

---
## 5. 트리 모델 비교

### 📚 트리 모델 진화

```
Decision Tree → Random Forest → XGBoost
   (단일 트리)    (여러 트리 투표)   (순차 학습)
```

- **Decision Tree**: 단순, 과적합 쉬움
- **Random Forest**: 여러 트리 앙상블, 안정적
- **XGBoost**: Gradient Boosting, 성능 최고

### 💻 실습 4: 3개 모델 비교

Decision Tree, Random Forest, XGBoost 성능을 비교하세요.

In [ ]:
# 💻 실습 4
results = []

# TODO: 3개 모델 정의
models = {
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': None,  # TODO: n_estimators=100
    'XGBoost': None        # TODO: n_estimators=100, eval_metric='logloss'
}

for name, model in models.items():
    if model is None:
        continue
    
    # TODO: 학습
    model.fit(X_train, y_train)
    
    # TODO: 예측
    pred = model.predict(X_test)
    proba = model.predict_proba(X_test)[:, 1]
    
    # 평가
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, pred),
        'Recall': recall_score(y_test, pred),
        'Precision': precision_score(y_test, pred, zero_division=0),
        'AUC': roc_auc_score(y_test, proba)
    })

result_df = pd.DataFrame(results)
result_df

In [ ]:
# 체크포인트 4
assert len(result_df) == 3, "3개 모델 결과 필요"
assert 'XGBoost' in result_df['Model'].values, "XGBoost 포함"

print("✅ 체크포인트 4 통과!")
print()
print("일반적으로 XGBoost ≥ RandomForest > DecisionTree")

---
## ✅ 최종 체크포인트

In [ ]:
print("="*50)
print("🎉 0-5 완료: ML 개념 + Sklearn")
print("="*50)
print()
print("배운 것:")
print("  - 분류 vs 회귀 (FDS = 분류)")
print("  - train_test_split (과적합 방지)")
print("  - fit → predict 패턴")
print("  - 평가 지표: Accuracy, Recall, Precision, AUC")
print("  - FDS에서 Recall이 중요 (사기 놓치면 안됨)")
print("  - 트리 모델: DecisionTree → RandomForest → XGBoost")
print()
print("다음: 0-6 모델 저장/튜닝/설명")